In [10]:
#Watershed on combined image region

import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

img = cv.imread('/home/rcardiff/ryan/ryan/test_files/combined_img-4.png')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
gray_inv = np.invert(gray)

kernel1 = np.array([[-1,-1, -1], [0, 0, 0], [1, 1, 1]], np.uint8)
opening = cv.morphologyEx(gray_inv,cv.MORPH_OPEN,kernel1, iterations = 3)
#opening = dilation. turns pixel to 1 if a pixel under the kernel is 1, increasing size of object and 
#helping connect broken objects

kernel2 = np.ones((3,3),np.uint8)
grad = cv.morphologyEx(opening,cv.MORPH_GRADIENT, kernel2, iterations =1)


# Marker labelling
ret, markers = cv.connectedComponents(grad)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
markers = cv.watershed(img,markers)

from skimage.measure import regionprops
print(len(regionprops(markers)))

196678


In [11]:
from pyteomics import mzxml, auxiliary
import matplotlib.patches as mpatches
import os
from statistics import median
from tqdm import tqdm_notebook as tqdm
import numpy as np

feature_data = [x.bbox for x in regionprops(markers)]
feature_list = [{} for x in regionprops(markers)]

min_mzs = np.array([x[0] for x in feature_data])
min_rts = np.array([x[1] for x in feature_data])
max_mzs = np.array([x[2] for x in feature_data])
max_rts = np.array([x[3] for x in feature_data])

min_mzs = np.round(150.0 + (min_mzs * .001), decimals = 4)
min_rts = np.round(min_rts * .02, decimals = 4)
max_mzs = np.round(150.0 + (max_mzs * .001), decimals = 4)
max_rts = np.round(max_rts * .02, decimals = 4)


num_features = len(min_mzs)

directory = "/home/rcardiff/ryan/ryan/meyer_raw_data/C18_neg/CL/"
in_files = [os.path.join(directory, x) for x in os.listdir(directory) if x.endswith('.mzXML')]
feature_list = [{} for r in regionprops(markers)]

#2D lists with (number files) x (number features) dimensions
running_feature_list_mz = [[[] for f in in_files] for r in regionprops(markers)]
running_feature_list_rt = [[[] for f in in_files] for r in regionprops(markers)]
running_feature_list_i = [[[] for f in in_files] for r in regionprops(markers)]



In [12]:
import pandas as pd
from numba import jit
import numpy as np
import sys
from pandarallel import pandarallel

## Define functions for scraping feature statistics

In [13]:
def make_file_df(fname):
    f_data = mzxml.read(fname)
    all_data =[(x['m/z array'], x['intensity array'], [float(x['retentionTime']) for y in x['m/z array']]) for x in f_data]
    mzs = np.concatenate([x[0] for x in all_data])
    intensities = np.concatenate([x[1] for x in all_data])
    rts = np.concatenate([x[2] for x in all_data])
    df_filedata = pd.DataFrame({'mz' : mzs, 'rt' : rts, 'intensity' : intensities})
    return(df_filedata)

def get_region_stats(feature, df_file):
    df_keep = df_file.query(f"mz>{feature['min_mz']} and mz<{feature['max_mz']} and rt>{feature['min_rt']} and rt<{feature['max_rt']}")
    if len(df_keep) == 0:
        return([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
    mz_min, mz_med, mz_max = np.percentile(df_keep['mz'], [0, 50, 100])
    rt_min, rt_med, rt_max = np.percentile(df_keep['rt'], [0, 50, 100])
    int_mean = np.mean(df_keep['intensity'])
    return([int_mean, mz_min, mz_max, mz_med, rt_min, rt_max, rt_med])

@jit(nopython=True)
def get_region_stats_vals(feature_vals, filedata_vals):
    kept_data = filedata_vals.T[np.where((filedata_vals[0] > feature_vals[0]) & (filedata_vals[0] < feature_vals[1]) & (filedata_vals[1] > feature_vals[2]) & (filedata_vals[1] < feature_vals[3]))]
    if len(kept_data) == 0:
        return([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])
    else:
        mz_min, mz_med, mz_max = np.percentile(kept_data.T[0], [0, 50, 100])
        rt_min, rt_med, rt_max = np.percentile(kept_data.T[1], [0, 50, 100])
        int_mean = np.mean(kept_data.T[2])
        return([int_mean, mz_min, mz_max, mz_med, rt_min, rt_max, rt_med])
                            
                            
def add_df_col(fname, df_features, test = True):
    if test:
        df_features = pd.DataFrame(df_features.iloc[0 : 1000])
    pandarallel.initialize(progress_bar=True, nb_workers=7)
    feature_box_cols = ['min_mz', 'max_mz', 'min_rt', 'max_rt', 'med_mz', 'med_rt', 'mean_int']
    df_file = make_file_df(fname)
    file_pref = os.path.basename(fname).strip('.mzXML')
                            
    out_col = df_features[feature_box_cols].parallel_apply(lambda x : get_region_stats_vals(x.values, df_file.values.T), axis = 1)
                            
    out_col = [x if not np.all(np.isnan(x)) else [] for x in out_col]
    df_features[file_pref] = out_col
    return(df_features)
    

## Create dataframe based on features from regionprops and add a column for each .mzXML file with feature statistics

In [15]:
df_features = pd.DataFrame({
    'min_mz' : min_mzs,
    'max_mz' : max_mzs,
    'min_rt' : min_rts,
    'max_rt' : max_rts
    'med_mz' :
    'med_rt' :
    'mean_int' :
})

for i_fname, fname in enumerate(tqdm(in_files)):
    print(f'Getting feature stats for {fname}')
#     df_features = add_df_col(fname, df_features, test = True)
    df_features = add_df_col(fname, df_features)

SyntaxError: invalid syntax (<ipython-input-15-771a057be52a>, line 6)

## Add feature ID column to dataframe and write to output tsv file

In [8]:
out_file_name = "df_features.tsv"
ndigits = len(str(len(df_features)))
feature_ids = [f"F{str(i).zfill(ndigits)}" for i in range(len(df_features))]
df_features.insert(0, 'feature_id', feature_ids)
df_features.to_csv(out_file_name, sep = '\t', index = False)